In [ ]:
from pathlib import Path
from typing import Literal
from glob import glob
import os
from IPython.display import display
from time import perf_counter
from collections import defaultdict

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from PIL import ImageDraw, ImageFont, Image

import cv2

---

---

In [ ]:
def display_image(image: np.ndarray, channel: int = None, save_as: Path = None):
    if channel is not None:
        # fig = plt.figure(figsize=(16, 9))
        single_channel_img = np.zeros_like(image)
        single_channel_img[..., channel] = image[..., channel]
        # plt.imshow(single_channel_img)
        # plt.axis('off')
        # plt.show()
        newimg = Image.fromarray(single_channel_img)
        display(newimg)
    else:
        newimg = Image.fromarray(image)
        display(newimg)

In [ ]:
images_path = Path("datasets/images")
backgrounds_path = Path("datasets/backgrounds")

save_path = Path('results')

In [ ]:
img_cv = cv2.imread(images_path / '2_Emilia_1.png', cv2.IMREAD_COLOR_RGB)
print(img_cv.shape)
display_image(img_cv, 0)

In [ ]:
img_back = cv2.imread(backgrounds_path / "2_bed.png", cv2.IMREAD_COLOR_RGB)
print(img_back.shape)
display_image(img_back)

In [ ]:
def resize_foreground_to_background(
    fg_img: np.ndarray,
    bg_img: np.ndarray,
    offset: bool = False
):
    H_bg, W_bg = bg_img.shape[:2]
    H_fg, W_fg = fg_img.shape[:2]

    scale_w = W_bg / W_fg
    scale_h = H_bg / H_fg
    scale = min(scale_w, scale_h)

    new_w = int(W_fg * scale)
    new_h = int(H_fg * scale)

    img_scaled = cv2.resize(fg_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    if offset:
        x_offset = (W_bg - new_w) // 2
        y_offset = (H_bg - new_h) // 2
        return img_scaled, x_offset, y_offset

    bg_color = fg_img[10, 10, :3].tolist()

    # --- Compute padding on both axes ---
    # Horizontal padding
    pad_left   = max((W_bg - new_w) // 2, 0)
    pad_right  = max(W_bg - new_w - pad_left, 0)

    # Vertical padding
    pad_top    = max((H_bg - new_h) // 2, 0)
    pad_bottom = max(H_bg - new_h - pad_top, 0)

    img_padded = cv2.copyMakeBorder(
        img_scaled,
        top=pad_top,
        bottom=pad_bottom,
        left=pad_left,
        right=pad_right,
        borderType=cv2.BORDER_CONSTANT,
        value=bg_color
    )

    return img_padded

In [ ]:
def resize_background_to_foreground(bg_img: np.ndarray, fg_img: np.ndarray) -> np.ndarray:
    H_fg, W_fg = fg_img.shape[:2]

    bg_resized = cv2.resize(
        bg_img,
        (W_fg, H_fg),
        interpolation=cv2.INTER_AREA
    )

    return bg_resized

In [ ]:
img_resized = resize_foreground_to_background(img_cv, img_back)
display_image(img_resized)

In [ ]:
# img_hsv = cv2.cvtColor(img_resized, cv2.COLOR_RGB2HSV)

# height, weight, _ = img_hsv.shape

# # define the range of hues to detect
# # - adjust these to detect different colours
# lower_green = np.array([55, 50, 50])
# upper_green = np.array([95, 255, 255])

# # create a mask that identifies the pixels in the range of hues
# mask = cv2.inRange(img_hsv, lower_green, upper_green)
# mask_inverted = cv2.bitwise_not(mask)


# # create a grey image and black out the masked area
# img_grey = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
# img_grey = cv2.bitwise_and(img_grey, img_grey, mask=mask_inverted)

# # black out unmasked area of original image
# image_masked = cv2.bitwise_and(img_resized, img_resized, mask=mask)

# # combine the two images for display
# img_grey = cv2.cvtColor(img_grey, cv2.COLOR_GRAY2RGB)
# img_combined = cv2.add(img_grey, image_masked)

# display_image(img_combined)

In [ ]:
def test_chroma(chromakey_func: callable, foregrounds_path: Path, backgrounds_path: Path, results_path: Path):
    
    func_results_dir = results_path / chromakey_func.__name__
    os.makedirs(func_results_dir, exist_ok=True)

    exec_time_log = []

    fg_images, bg_images = [], []
    for img_list, img_dir_path in zip((fg_images, bg_images), (foregrounds_path, backgrounds_path)):
        for img_path in glob(f'{img_dir_path.as_posix()}/*', recursive=True):
            if img_path.endswith(('png', 'jpg', 'jpeg')):
                img_list.append(img_path)

    fg_images = sorted(fg_images)
    bg_images = sorted(bg_images)

    _get_name = lambda pth: pth.split('/')[-1].split('.')[0]

    for fg_img_pth, bg_img_pth in zip(fg_images, bg_images):
        fg_img = cv2.imread(fg_img_pth, cv2.IMREAD_COLOR_RGB)
        bg_img = cv2.imread(bg_img_pth, cv2.IMREAD_COLOR_RGB)

        fg_img_resized = resize_foreground_to_background(fg_img, bg_img)
        
        start_time = perf_counter()
        img_chroma = chromakey_func(fg_img_resized, bg_img, 'hsv')
        end_time = perf_counter()
        exec_time_log.append(end_time - start_time)

        img_name = _get_name(fg_img_pth) + '_' + _get_name(bg_img_pth) + '.png'
        # cv2.imwrite(results_path / img_name, cv2.cvtColor(img_chroma, cv2.COLOR_RGB2BGR))
        Image.fromarray(img_chroma).save(func_results_dir / img_name)

    return exec_time_log

In [ ]:
def test_chroma_hard(chromakey_func: callable, foregrounds_path: Path, backgrounds_path: Path, results_path: Path):
    
    func_results_dir = results_path / chromakey_func.__name__
    os.makedirs(func_results_dir, exist_ok=True)

    exec_time_log = []

    fg_images, bg_images = [], []
    for img_list, img_dir_path in zip((fg_images, bg_images), (foregrounds_path, backgrounds_path)):
        for img_path in glob(f'{img_dir_path.as_posix()}/*', recursive=True):
            if img_path.endswith(('png', 'jpg', 'jpeg')):
                img_list.append(img_path)

    fg_images = sorted(fg_images)
    bg_images = sorted(bg_images)

    _get_name = lambda pth: pth.split('/')[-1].split('.')[0]

    for fg_img_pth in fg_images:
        for bg_img_pth in bg_images:
            fg_img = cv2.imread(fg_img_pth, cv2.IMREAD_COLOR_RGB)
            bg_img = cv2.imread(bg_img_pth, cv2.IMREAD_COLOR_RGB)

            fg_img_resized = resize_foreground_to_background(fg_img, bg_img)
            
            start_time = perf_counter()
            img_chroma = chromakey_func(fg_img_resized, bg_img, 'hsv')
            end_time = perf_counter()
            exec_time_log.append(end_time - start_time)

            img_name = _get_name(fg_img_pth) + '_' + _get_name(bg_img_pth) + '.png'
            # cv2.imwrite(results_path / img_name, cv2.cvtColor(img_chroma, cv2.COLOR_RGB2BGR))
            Image.fromarray(img_chroma).save(func_results_dir / img_name)

    return exec_time_log

## Manual implementation

### Super Naive

In [ ]:
def apply_chromakey_naive(    
    fg_img: np.ndarray, bg_img: np.ndarray, # rgb
    color_model: str = 'rgb',               # not used, placeholder
    sample_bg_color: bool = False,          # not used, placeholder
    feathering: tuple[int, int] = None,     # not used, placeholder
) -> np.ndarray:
    
    front_cp = deepcopy(fg_img)
    mask = (fg_img[:, :, 1] > 200) & (fg_img[:, :, 2] < 100) & (fg_img[:, :, 0] < 100)
    front_cp[mask] = bg_img[mask]

    # newimg = Image.fromarray(front_cp)
    # display(newimg)
    # newimg.save('output.jpg')

    return front_cp

In [ ]:
img_chroma = apply_chromakey_naive(img_resized, img_back)
display_image(img_chroma)

In [ ]:
Image.fromarray(img_chroma).save(save_path / 'emilia_naive.png')

---

## Naive upgraded

In [ ]:
def find_borders(img):

    def conv(img, kernel):

        kh, kw = kernel.shape
        ph, pw = kh//2, kw//2
        padded = np.pad(img, ((ph, ph), (pw, pw)), mode='edge')
        window = np.lib.stride_tricks.sliding_window_view(padded, (kh, kw))
        filtered = (window * kernel).sum(axis=(-2, -1))
        return filtered

    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    sobel_y = np.array([[-1, -2, -1], [ 0,  0,  0], [ 1,  2,  1]], dtype=np.float32)
    grad_x = conv(img, sobel_x)
    grad_y = conv(img, sobel_y)
    grad_mag = np.hypot(grad_x, grad_y)
    return grad_mag

def median_filter_2d(arr, size=3):
    pad = size // 2
    padded = np.pad(arr, pad, mode='edge')
    window = np.lib.stride_tricks.sliding_window_view(padded, (size, size))
    arr = np.median(window, axis=(-2,-1))
    return arr


def apply_chromakey_naive_plus(    
    fg_img: np.ndarray, bg_img: np.ndarray, # rgb
    color_model: str = 'rgb',               # not used, placeholder
    sample_bg_color: bool = False,          # not used, placeholder
    feathering: tuple[int, int] = None,     # not used, placeholder
) -> np.ndarray:
    
    threshold = 10
    front_cp = deepcopy(fg_img)
    G = fg_img[:, :, 1].astype(np.float32)
    R = fg_img[:, :, 0].astype(np.float32)
    B = fg_img[:, :, 2].astype(np.float32)

    greenness = np.clip((G - np.maximum(R, B)) / 255.0, 0, 1)
    mask = greenness > 0.3
    filtered = find_borders(mask)
    grad_norm = np.clip(filtered / filtered.max() * 255, 0, 255).astype(np.uint8)
    grad_clean = median_filter_2d(grad_norm, size=3)
    border_mask = grad_clean > threshold
    mask = mask + border_mask
    front_cp[mask] = bg_img[mask]
    # newimg = Image.fromarray(front_cp)
    # display(newimg)
    return front_cp

In [ ]:
img_chroma = apply_chromakey_naive_plus(img_resized, img_back)
display_image(img_chroma)

In [ ]:
Image.fromarray(img_chroma).save(save_path / 'emilia_naive_plus.png')

---

## OpenCV implementation

In [ ]:
def apply_chromakey_opencv(
    fg_img, bg_img, 
    color_model: Literal['rgb', 'hsv'] = 'hsv',
    sample_bg_color: bool = False,
    feathering: tuple[int, int] = None,
) -> np.ndarray:

    if color_model == 'rgb':
        if not sample_bg_color:
            lower_green = np.array([0, 150, 0])
            upper_green = np.array([120, 255, 120])
        else:
            lower_green = upper_green = fg_img[10, 10, :3]
        fg_cpy = fg_img
    else:
        fg_cpy = cv2.cvtColor(fg_img, cv2.COLOR_RGB2HSV)
        if not sample_bg_color:
            lower_green = np.array([45, 120, 120])
            upper_green = np.array([85, 255, 255])
        else:
            lower_green = upper_green = fg_cpy[10, 10, :3]

    height, width, _ = fg_img.shape

    background_mask = cv2.inRange(fg_cpy, lower_green, upper_green)
    foreground_mask = cv2.bitwise_not(background_mask)

    # extract the set of contours around the foreground mask and then the
    # largest contour as the foreground object of interest. Update the
    # foreground mask with all the pixels inside this contour
    contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if (len(contours) > 0):
        largest_contour = max(contours, key=cv2.contourArea)
        foreground_mask_object = np.zeros((height, width))
        cv2.fillPoly(foreground_mask_object, [largest_contour], (255))
        foreground_mask = foreground_mask_object

    # recompute the background mask based on the updated foreground mask
    background_mask = ((np.ones((height, width)) * 255) - foreground_mask).astype(np.uint8)

    # construct 3-channel RGB feathered masks for blending
    foreground_mask = foreground_mask.astype(np.float32) / 255.
    background_mask = 1 - foreground_mask

    if feathering is not None:
        foreground_mask = cv2.blur(foreground_mask, feathering)
        background_mask = cv2.blur(background_mask, feathering)

    fg3 = cv2.merge([foreground_mask, foreground_mask, foreground_mask])
    bg3 = cv2.merge([background_mask, background_mask, background_mask])
    # fg3 = np.expand_dims(foreground_mask, -1)
    # bg3 = np.expand_dims(background_mask, -1)

    # combine current camera image with new background via feathered blending
    chroma_key_image = ((bg3 * bg_img.astype(np.float32)) + (fg3 * fg_img.astype(np.float32))).astype(np.uint8)

    return chroma_key_image

In [ ]:
img_chroma = apply_chromakey_opencv(img_resized, img_back, 'hsv', sample_bg_color=False)
display_image(img_chroma)

In [ ]:
cv2.imwrite(save_path / 'emilia_cv.png', cv2.cvtColor(img_chroma, cv2.COLOR_RGB2BGR))

---

## Tests

In [ ]:
logs = defaultdict(list)

In [ ]:
for i in range(5):
    start_time = perf_counter()
    logs['naive'].extend(test_chroma(apply_chromakey_naive, images_path, backgrounds_path, save_path))
    logs['naive_plus'].extend(test_chroma(apply_chromakey_naive_plus, images_path, backgrounds_path, save_path))
    logs['opencv'].extend(test_chroma(apply_chromakey_opencv, images_path, backgrounds_path, save_path))
    end_time = perf_counter()
    print(f'Iter {i + 1}. Elapsed time: {end_time - start_time:.2f} sec.')

In [ ]:
start_time = perf_counter()
logs['naive'].extend(test_chroma_hard(apply_chromakey_naive, images_path, backgrounds_path, save_path))
s1 = perf_counter()
print(f'"Naive". Elapsed time: {s1 - start_time:.2f} sec.')

logs['naive_plus'].extend(test_chroma_hard(apply_chromakey_naive_plus, images_path, backgrounds_path, save_path))
s2 = perf_counter()
print(f'"Naive plus". Elapsed time: {s2 - s1:.2f} sec.')

logs['opencv'].extend(test_chroma_hard(apply_chromakey_opencv, images_path, backgrounds_path, save_path))
end_time = perf_counter()
print(f'"OpenCV". Elapsed time: {end_time - s2:.2f} sec.')

print(f'Test time: {end_time - start_time:.2f} sec.')

In [ ]:
def plot_logs_histograms(logs, nbins=30, title="Distribution of exec. time by method", save_path=None):
    records = []
    for key, values in logs.items():
        for v in values:
            records.append({"method": key, "value": v})

    fig = px.histogram(
        records,
        x="value",
        color="method",
        nbins=nbins,
        barmode="group",
        opacity=0.5,
        title=title,
    )

    fig.update_layout(
        xaxis_title="Time, sec.",
        yaxis_title="Count",
        legend_title="Method",
    )

    fig.show()

    if save_path:
        fig.write_image(save_path, scale=3)
        print(f"Histogram saved to {save_path}")


def plot_cumulative_lines(logs, title="Cumulative exec. time by method", save_path=None):
    fig = go.Figure()

    for method, values in logs.items():
        if not values:
            continue  # skip empty lists

        # Compute cumulative sum
        cumulative = np.cumsum(values)

        # Add line to figure
        fig.add_trace(go.Scatter(
            x=list(range(1, len(cumulative) + 1)),
            y=cumulative,
            mode='lines',
            name=method
        ))

    fig.update_layout(
        title=title,
        xaxis_title="No. of test",
        yaxis_title="Cumulative time, sec.",
        legend_title="Method",
    )

    fig.show()
    
    if save_path:
        fig.write_image(save_path, scale=3)
        print(f"Line chart saved to {save_path}")

In [ ]:
plot_logs_histograms(logs, save_path = save_path / 'hist.png')

In [ ]:
plot_cumulative_lines(logs, save_path = save_path / 'line.png')